In [4]:
import keras
import numpy as np
import os

In [12]:
D=[]
check=[]
for folder in ["ok","nok"]:
    for file in os.listdir(folder):
        if(".npy" in file):
            spectogram = np.load(folder+"/"+file)
            if spectogram.shape != (128, 128): continue
            D.append((spectogram, folder))
            check.append(spectogram)
print(len(D),"done")

24 done


In [19]:
print(np.amin(keras.utils.normalize(check,axis=0,order=2)),np.amax(keras.utils.normalize(check,axis=0,order=2)))

0.0003103770592151443 0.99630420051724


In [9]:
dataset = keras.utils.normalize(D)
print(len(D))
print(len(D[0]))
print(len(D[0][0]))
random.shuffle(dataset)

train = dataset[:600]
test = dataset[600:]

x_train, y_train = zip(*train)
x_test, y_test = zip(*test)

# Reshape for CNN input
x_train = np.array([x.reshape( (128, 128, 1) ) for x in x_train])
x_test = np.array([x.reshape( (128, 128, 1) ) for x in x_test])

# One-Hot encoding for classes
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)
print(y_test)

AttributeError: 'str' object has no attribute 'conjugate'